In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gru_96/keras/default/1/model_96.h5
/kaggle/input/amazon-dataset/data.npz
/kaggle/input/emb/keras/default/1/fully_pruned_model_latest.h5
/kaggle/input/gru_90/keras/default/1/model_90.h5
/kaggle/input/gru_pruned_model/keras/default/1/model_60.h5


In [4]:
import tensorflow as tf

In [5]:
# Load the pruned model
pruned_model = tf.keras.models.load_model("/kaggle/input/emb/keras/default/1/fully_pruned_model_latest.h5")
pruned_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 170, 64)             │       1,920,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 170, 128)            │          49,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │          74,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,069,251 (7.89 MB)

 Trainable params: 2,069,249 (7.89 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [6]:
import tensorflow as tf
import numpy as np

def count_trainable_zero_nonzero_params(model):
    total_params = 0
    zero_params = 0
    nonzero_params = 0

    # Iterate over trainable variables of the model
    for var in model.trainable_variables:
        total = tf.size(var).numpy()  # Total number of parameters in the variable
        zero = np.sum(var.numpy() == 0)  # Count zero parameters
        nonzero = total - zero  # Non-zero parameters

        total_params += total
        zero_params += zero
        nonzero_params += nonzero

    return total_params, zero_params, nonzero_params

# Example Usage
total, zero, nonzero = count_trainable_zero_nonzero_params(pruned_model)
print(f"Total Trainable Parameters: {total}")
print(f"Zero Trainable Parameters: {zero}")
print(f"Non-Zero Trainable Parameters: {nonzero}")


Total Trainable Parameters: 2069249
Zero Trainable Parameters: 146608
Non-Zero Trainable Parameters: 1922641


In [7]:
def layerwise_trainable_zero_nonzero_params(model):
    for layer in model.layers:
        if hasattr(layer, 'trainable_variables'):
            for var in layer.trainable_variables:
                total = tf.size(var).numpy()
                zero = np.sum(var.numpy() == 0)
                nonzero = total - zero
                print(f"Layer: {layer.name} | Total: {total} | Zero: {zero} | Non-Zero: {nonzero}")

# Example Usage
layerwise_trainable_zero_nonzero_params(pruned_model)


Layer: embedding | Total: 1920000 | Zero: 3392 | Non-Zero: 1916608
Layer: bidirectional | Total: 12288 | Zero: 11505 | Non-Zero: 783
Layer: bidirectional | Total: 12288 | Zero: 11488 | Non-Zero: 800
Layer: bidirectional | Total: 384 | Zero: 370 | Non-Zero: 14
Layer: bidirectional | Total: 12288 | Zero: 11346 | Non-Zero: 942
Layer: bidirectional | Total: 12288 | Zero: 11489 | Non-Zero: 799
Layer: bidirectional | Total: 384 | Zero: 358 | Non-Zero: 26
Layer: bidirectional_1 | Total: 24576 | Zero: 24058 | Non-Zero: 518
Layer: bidirectional_1 | Total: 12288 | Zero: 11971 | Non-Zero: 317
Layer: bidirectional_1 | Total: 384 | Zero: 379 | Non-Zero: 5
Layer: bidirectional_1 | Total: 24576 | Zero: 23691 | Non-Zero: 885
Layer: bidirectional_1 | Total: 12288 | Zero: 11709 | Non-Zero: 579
Layer: bidirectional_1 | Total: 384 | Zero: 358 | Non-Zero: 26
Layer: dense | Total: 16384 | Zero: 16288 | Non-Zero: 96
Layer: dense | Total: 128 | Zero: 127 | Non-Zero: 1
Layer: dense_1 | Total: 8192 | Zero: 8023

In [ ]:
# import tensorflow as tf

In [4]:
def create_bigru_dense_zero_mask(model):
    """
    Creates a mask for BiGRU and Dense layers where weights are zero.
    """
    masks = []
    for layer in model.layers:
        if isinstance(layer, (tf.keras.layers.Dense, tf.keras.layers.Bidirectional)):
            layer_masks = []
            for weight in layer.trainable_weights:
                mask = tf.cast(weight != 0, tf.float32)  # Mask: 1 for non-zero, 0 for zero
                layer_masks.append(mask)
            masks.append(layer_masks)
    return masks


In [5]:
def prune_embedding_lth(model, prune_ratio):
    """
    Prunes the Embedding layer using the Lottery Ticket Hypothesis.
    """
    embedding_layer = None
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Embedding):
            embedding_layer = layer
            break

    if embedding_layer is None:
        raise ValueError("No embedding layer found in the model.")

    # Get current embedding weights
    embedding_weights = embedding_layer.get_weights()[0]
    
    # Compute global threshold
    flat_weights = np.abs(embedding_weights.flatten())
    threshold = np.percentile(flat_weights, prune_ratio)

    # Create mask
    mask = tf.cast(tf.abs(embedding_weights) >= threshold, tf.float32)

    # Apply mask to embedding weights
    pruned_weights = embedding_weights * mask
    embedding_layer.set_weights([pruned_weights])

    return model, mask


In [6]:
def apply_bigru_dense_mask(model, masks):
    """
    Re-applies the zero mask to BiGRU and Dense layers after training updates.
    """
    idx = 0
    for layer in model.layers:
        if isinstance(layer, (tf.keras.layers.Dense, tf.keras.layers.Bidirectional)):
            for i, weight in enumerate(layer.trainable_weights):
                weight.assign(weight * masks[idx][i])  # Apply mask to retain zero weights
            idx += 1


In [7]:
def rewind_embedding_preserving_zeros(model, initial_embedding_weights, embedding_mask):
    """
    Rewinds the embedding layer to its initial values while preserving zeroed-out weights.
    """
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Embedding):
            restored_weights = initial_embedding_weights * embedding_mask  # Apply mask
            layer.set_weights([restored_weights])
            break  # Since there's only one embedding layer, exit loop
    return model


In [8]:
import tensorflow as tf

In [9]:
model = tf.keras.models.load_model("/kaggle/input/gru_96/keras/default/1/model_96.h5")
# 1️⃣ Create mask for BiGRU and Dense layers (zero weights)
bigru_dense_masks = create_bigru_dense_zero_mask(model)

# 2️⃣ Prune Embedding layer using LTH
prune_ratio = 96  # Adjust pruning percentage
model, embedding_mask = prune_embedding_lth(model, prune_ratio=prune_ratio)

# Save original embedding weights before pruning
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Embedding):
        initial_embedding_weights = layer.get_weights()[0]
        break

# 3️⃣ Rewind Embedding while keeping zero weights unchanged
model = rewind_embedding_preserving_zeros(model, initial_embedding_weights, embedding_mask)

data = np.load("/kaggle/input/amazon-dataset/data.npz")

def get_accuracy(model, val_dataset):
    # Initialize accuracy metric
    accuracy = tf.keras.metrics.BinaryAccuracy()
    # Iterate over validation dataset
    for x, y in val_dataset:
        # Predict and update accuracy state
        y_pred = model(x)
        accuracy.update_state(y, y_pred)
    # Return final accuracy result
    return accuracy.result().numpy()

# Convert to tensors
X_train = tf.convert_to_tensor(data["X_train"], dtype=tf.float32)
y_train = tf.convert_to_tensor(data["y_train"], dtype=tf.float32)
X_val = tf.convert_to_tensor(data["X_val"], dtype=tf.float32)
y_val = tf.convert_to_tensor(data["y_val"], dtype=tf.float32)

# Smaller validation subset
val_sample_size = 8000
X_val = X_val[:val_sample_size]
y_val = y_val[:val_sample_size]
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(64)

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(10000).batch(1900)

learning_rate = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.keras.losses.BinaryCrossentropy()

# 4️⃣ Train while applying BiGRU & Dense zero masks
num_epochs = 3
for epoch in range(num_epochs):
    for x_batch, y_batch in train_ds:
        with tf.GradientTape() as tape:
            y_pred = model(x_batch, training=True)
            loss = loss_fn(y_batch, y_pred)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # Reapply BiGRU & Dense masks after each update
        apply_bigru_dense_mask(model, bigru_dense_masks)

    print(f"Epoch {epoch+1}/{num_epochs} completed.")

# Evaluate performance
val_accuracy = get_accuracy(model, val_dataset)
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Save the final pruned model
model.save("/kaggle/working/fully_pruned_model00.h5")


Epoch 1/3 completed.
Epoch 2/3 completed.
Epoch 3/3 completed.
Validation Accuracy: 0.9308


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 170, 64)             │       1,920,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 170, 128)            │          49,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │          74,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,069,251 (7.89 MB)

 Trainable params: 2,069,249 (7.89 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [11]:
import tensorflow as tf
import numpy as np

def count_trainable_zero_nonzero_params(model):
    total_params = 0
    zero_params = 0
    nonzero_params = 0

    # Iterate over trainable variables of the model
    for var in model.trainable_variables:
        total = tf.size(var).numpy()  # Total number of parameters in the variable
        zero = np.sum(var.numpy() == 0)  # Count zero parameters
        nonzero = total - zero  # Non-zero parameters

        total_params += total
        zero_params += zero
        nonzero_params += nonzero

    return total_params, zero_params, nonzero_params

# Example Usage
total, zero, nonzero = count_trainable_zero_nonzero_params(model)
print(f"Total Trainable Parameters: {total}")
print(f"Zero Trainable Parameters: {zero}")
print(f"Non-Zero Trainable Parameters: {nonzero}")


Total Trainable Parameters: 2069249
Zero Trainable Parameters: 146608
Non-Zero Trainable Parameters: 1922641


In [12]:
def layerwise_trainable_zero_nonzero_params(model):
    for layer in model.layers:
        if hasattr(layer, 'trainable_variables'):
            for var in layer.trainable_variables:
                total = tf.size(var).numpy()
                zero = np.sum(var.numpy() == 0)
                nonzero = total - zero
                print(f"Layer: {layer.name} | Total: {total} | Zero: {zero} | Non-Zero: {nonzero}")

# Example Usage
layerwise_trainable_zero_nonzero_params(model)


Layer: embedding | Total: 1920000 | Zero: 3392 | Non-Zero: 1916608
Layer: bidirectional | Total: 12288 | Zero: 11505 | Non-Zero: 783
Layer: bidirectional | Total: 12288 | Zero: 11488 | Non-Zero: 800
Layer: bidirectional | Total: 384 | Zero: 370 | Non-Zero: 14
Layer: bidirectional | Total: 12288 | Zero: 11346 | Non-Zero: 942
Layer: bidirectional | Total: 12288 | Zero: 11489 | Non-Zero: 799
Layer: bidirectional | Total: 384 | Zero: 358 | Non-Zero: 26
Layer: bidirectional_1 | Total: 24576 | Zero: 24058 | Non-Zero: 518
Layer: bidirectional_1 | Total: 12288 | Zero: 11971 | Non-Zero: 317
Layer: bidirectional_1 | Total: 384 | Zero: 379 | Non-Zero: 5
Layer: bidirectional_1 | Total: 24576 | Zero: 23691 | Non-Zero: 885
Layer: bidirectional_1 | Total: 12288 | Zero: 11709 | Non-Zero: 579
Layer: bidirectional_1 | Total: 384 | Zero: 358 | Non-Zero: 26
Layer: dense | Total: 16384 | Zero: 16288 | Non-Zero: 96
Layer: dense | Total: 128 | Zero: 127 | Non-Zero: 1
Layer: dense_1 | Total: 8192 | Zero: 8023